In [33]:
# Importing necessary libraries for training and visualization
import numpy as np
import pandas as pd
import torch
import torchvision
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torch.autograd import Variable
import tensorflow as tf
from sklearn import preprocessing

import matplotlib.pyplot as plt
import tqdm



train_set = pd.read_csv("/Users/cszilagyi2019/Desktop/All Coding/Natural Disaster Tweets/train.csv")
test_set = pd.read_csv("/Users/cszilagyi2019/Desktop/All Coding/Natural Disaster Tweets/test.csv")

In [3]:
#Setting up Training/Validation/Testing through the dataloaders

#Hyperparameters
batch_size = 1
validation_split = 0.2
num_epochs = 20

# Create validation split by taking a percentage of the training set:
dataset_size = len(train_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

train_indices, val_indices = indices[split:], indices[:split]

# Use the SubsetRandomSampler to randomly sample the training dataset for 
# training and validation data. We will feed this into the dataloader below.
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Create dataloader objects - will be used during training and inference
# to iterate over the data.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, \
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, \
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                                          shuffle=False)

In [4]:
# Creating the Neural Network Class

class Net(nn.Module):

  def __init__(self, input_size, hidden_size, num_classes):

    super(Net, self).__init__()

    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.output = nn.Linear(hidden_size, num_classes)
    
    # Applying activation function
    self.sigmoid = nn.Sigmoid()

    
  #Forward pass
  def forward(self, x):

    out = self.fc1(x)
    out = self.sigmoid(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    out = self.output(out)
    #Will use cross entropy
    return out

In [44]:
#Making the dataset

class myDataSet(Dataset):
    
    def __init__(self, file):
        
        #Reading the file into variables
        self.x = file.iloc[:, [2,4,5,6,7,9,11]]
        self.y = file.iloc[1:892, 1]
        
        mapping = {'male': 0, 'female': 1, 'S': 0, 'C': 1, "Q": 2}
        self.x.replace({'Sex': mapping, 'Embarked': mapping})
        
        
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.x.iloc[idx], self.y.iloc[idx]
        

In [45]:
#Setting up Training/Validation/Testing through the dataloaders

#Hyperparameters
batch_size = 1
validation_split = 0.2
num_epochs = 20

# Create validation split by taking a percentage of the training set:
dataset_size = len(train_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

train_indices, val_indices = indices[split:], indices[:split]

# Use the SubsetRandomSampler to randomly sample the training dataset for 
# training and validation data. We will feed this into the dataloader below.
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)


#Making the datasets
train_dataset = myDataSet(train_set)


# Create dataloader objects - will be used during training and inference
# to iterate over the data.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, \
                                         sampler=val_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, \
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                                          shuffle=False)

In [30]:
#Making the neural network
net = Net(7, 10, 2)

#Cross Entropy Loss/Adam Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [46]:
#Training loop

running_loss = 0.0

for epoch in range(num_epochs):

  for i, (data, survived) in enumerate(train_loader):
    print(data)
    # Clear the gradients before performing backpropagation
    optimizer.zero_grad()
    
    x_train = torch.tensor(data)
    
    # Perform the forward pass - this call of net calls the forward() fn
    outputs = net(data)
    
    
    
    # Compute loss on the output of the forward pass and desired label
    loss = criterion(outputs, labels)
    # Compute the gradients with respect to the loss function
    loss.backward()      
    # Update the weights in the neural network using the optimizer/backprop
    optimizer.step()

    # Track losses for plotting later
    loss_tracker.append(loss.data) 

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.series.Series'>

In [21]:
# Function for getting accuracy, adapted from: 
# https://towardsdatascience.com/a-simple-starter-guide-to-build-a-neural-network-3c2cf07b8d7c

def get_accuracy(loader, my_net):
  correct = 0
  total = 0
  for images, labels in loader:
      images = Variable(images.view(-1, 28*28))
      outputs = my_net(images)
      _, predicted = torch.max(outputs.data, 1)  
      total += labels.size(0)               
      correct += (predicted == labels).sum()

  return 100 * correct / total